In [1]:
from bs4 import BeautifulSoup, SoupStrainer #HTML parsing
import urllib.request #aufrufen von URLs
from time import sleep #damit legen wir den Scraper schlafen
import json #lesen und schreiben von JSON-Dateien
from datetime import datetime #um den Daten Timestamps zu geben
import re #regular expressions
import os #Dateipfade erstellen und lesen
import pandas as pd #Datenanalyse und -manipulation

In [ ]:
folders = ["data/visited/","data/autos/"]
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)
        print(path, "erstellt.")
    else:
        print(folder,"existiert bereits")

In [ ]:
path_to_visited_urls = "data/visited/visited_urls.json"
if not os.path.isfile(path_to_visited_urls):
    with open(path_to_visited_urls,"w") as file:
        json.dump([],file)

In [ ]:

countries = {"Deutschland": "D"}

In [ ]:
car_counter=1
cycle_counter=0

In [ ]:
while True:
    with open(path_to_visited_urls) as file:
        visited_urls = json.load(file)
    
    if len(visited_urls) > 100000:
        visited_urls = []
    
    multiple_cars_dict = {}
    
    cycle_counter+=1
    for country in countries:
        
        car_URLs = []
        
        for page in range(1,21):
            try:
                url = 'https://www.autoscout24.de/lst/?sort=age&desc=1&ustate=N%2CU&size=20&page='+str(page)+ '&cy=' + countries[country] +'&atype=C&'
                only_a_tags = SoupStrainer("a")
                soup = BeautifulSoup(urllib.request.urlopen(url).read(),'lxml', parse_only=only_a_tags)
            except Exception as e:
                print("Übersicht: " + str(e) +" "*50, end="\r")
                pass
            for link in soup.find_all("a"):
                if r"/angebote/" in str(link.get("href")):
                    car_URLs.append(link.get("href"))
            car_URLs_unique = [car for car in list(set(car_URLs)) if car not in visited_urls]
            
            print(f'Lauf {cycle_counter} | {country} | Seite {page} | {len(car_URLs_unique)} neue URLs', end="\r")
        print("")
        if len(car_URLs_unique)>0:
            for URL in car_URLs_unique:
                print(f'Lauf {cycle_counter} | {country} | Auto {car_counter}'+' '*50, end="\r")
                try:
                    car_counter+=1
                    car_dict = {}
                    car_dict["country"] = country
                    car_dict["date"] = str(datetime.now())                    
                    car = BeautifulSoup(urllib.request.urlopen('https://www.autoscout24.de'+URL).read(),'lxml')
                    
                    for key, value in zip(car.find_all("dt"),car.find_all("dd")):
                        car_dict[key.text.replace("\n","")] = value.text.replace("\n","")
                    car_dict["haendler"] = car.find("div",attrs={"class":"cldt-vendor-contact-box",
                                                                 "data-vendor-type":"dealer"}) != None
                    car_dict["privat"] = car.find("div",attrs={"class":"cldt-vendor-contact-box",
                                                               "data-vendor-type":"privateseller"}) != None
                    car_dict["ort"] = car.find("div",attrs={"class":"sc-grid-col-12",
                                                            "data-item-name":"vendor-contact-city"}).text
                    
                    car_dict["price"] =  "".join(re.findall(r'[0-9]+',car.find("div",attrs={"class":"cldt-price"}).text))
                    
                    ausstattung = []
                    for i in car.find_all("div",attrs={"class":"cldt-equipment-block sc-grid-col-3 sc-grid-col-m-4 sc-grid-col-s-12 sc-pull-left"}):
                        for span in i.find_all("span"):
                            ausstattung.append(i.text)
                    ausstattung2 = []
                    for element in list(set(ausstattung)):
                        austattung_liste = element.split("\n")
                        ausstattung2.extend(austattung_liste)
                    car_dict["ausstattung_liste"] = sorted(list(set(ausstattung2)))
                    multiple_cars_dict[URL] = car_dict
                    visited_urls.append(URL)
                except Exception as e:
                    print("Detailseite: " + str(e) + " "*50)
                    pass
            print("")
        
        else:
            print("\U0001F634")
            sleep(60)
    
    if len(multiple_cars_dict)>0:
        df = pd.DataFrame(multiple_cars_dict).T
        df.to_csv("data/autos/"+re.sub("[.,:,-, ]","_",str(datetime.now()))+".csv",sep=";",index_label="url")
    else:
        print("Keine Daten")
    with open("data/visited/visited_urls.json", "w") as file:
        json.dump(visited_urls, file)

In [ ]:
URL

In [8]:
car = BeautifulSoup(urllib.request.urlopen(my_url).read(),'lxml')

In [9]:
car

<!DOCTYPE html>
<html lang="de-DE"><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><title>Mercedes-Benz C 220 Kombi in Schwarz gebraucht in Wedel für € 11.900,-</title><link as="image" href="https://prod.pictures.autoscout24.net/listing-images/47d79386-2bfb-442e-bca2-a51b4902c780_737fb759-6227-46b7-ba3a-0d955b3c3866.jpg/360x270.webp" media="(max-width: 360px)" rel="preload"/><link as="image" href="https://prod.pictures.autoscout24.net/listing-images/47d79386-2bfb-442e-bca2-a51b4902c780_737fb759-6227-46b7-ba3a-0d955b3c3866.jpg/480x360.webp" media="(min-width: 361px) and (max-width: 480px)" rel="preload"/><link as="image" href="https://prod.pictures.autoscout24.net/listing-images/47d79386-2bfb-442e-bca2-a51b4902c780_737fb759-6227-46b7-ba3a-0d955b3c3866.jpg/720x540.webp" media="(max-width: 720px) and (-webkit-max-device-pixel-ratio: 1), (min-width: 481px) and (max-width: 719px) and (-webkit-min-device-pixel-ratio: 1.01), (min-width: 1024px)" rel="preload"/

In [10]:
from urllib.request import urlopen
import html5lib

with urlopen(my_url) as f:
    document = html5lib.parse(f, transport_encoding=f.info().get_content_charset())


In [11]:
print(document)

<Element '{http://www.w3.org/1999/xhtml}html' at 0x127944a40>


In [ ]:
for key, value in zip(car.find_all("dt"),car.find_all("dd")):
                        car_dict[key.text.replace("\n","")] = value.text.replace("\n","")

In [13]:
key_list = []
value_list = []
for key, value in zip(car.find_all('dt'), car.find_all('dd')):
    key_list.append(key.text)
    value_list.append(value.text)

print(key_list)
print(value_list)

['Karosserieform', 'Fahrzeugart', 'Antriebsart', 'Sitzplätze', 'Türen', 'Länderversion', 'Angebotsnummer', 'Schlüsselnummer', 'Garantie', 'Kilometerstand', 'Erstzulassung', 'Baujahr', 'HU', 'Scheckheftgepflegt', 'Nichtraucherfahrzeug', 'Leistung', 'Getriebe', 'Hubraum', 'Gänge', 'Zylinder', 'Leergewicht', 'Kraftstoff', 'Kraftstoffverbrauch2,8', 'CO₂-Emissionen2,8', 'Energieeffizienzklasse8', 'CO₂-Effizienz', 'Schadstoffklasse', 'Umweltplakette', 'Komfort', 'Unterhaltung/Media', 'Sicherheit', 'Extras', 'Außenfarbe', 'Farbe laut Hersteller', 'Lackierung', 'Farbe der Innenausstattung', 'Innenausstattung', 'Anzahlung', 'Überführungskosten', 'Zulassungskosten', 'Gesamt, einmalig', 'Leasinggesamtbetrag', 'Vertragsart', 'Fahrleistung p.a.', 'Leasingfaktor', 'Laufzeit', 'Monatliche Rate']
['Kombi', 'Gebraucht', 'Heck', '5', '5', 'Deutsche Ausführung', 'WE-03191', '1313/AGX', 'Ja', '176.212 km', '05/2014', '2014', 'Neu', 'Ja', 'Ja', '125 kW (170 PS)', 'Schaltgetriebe', '2.143 cm³', '6', '4', '1

In [7]:
print(car.find("div",attrs={"class":"cldt-vendor-contact-box", "data-vendor-type":"privateseller"}))

None


In [12]:
price_category = car.find('div', attrs={'class': "Price_priceCategoryContainer__v1_wn"})
price_category.getText()

'Sehr guter Preis'

In [15]:
type(car)

bs4.BeautifulSoup

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

my_url01 = 'https://www.autoscout24.de/angebote/mercedes-benz-c-220-t-blueefficiency-edition-cdi-diesel-schwarz-47d79386-2bfb-442e-bca2-a51b4902c780?sort=standard&desc=0&lastSeenGuidPresent=false&cldtidx=1&position=1&search_id=17ucufqxic&source_otp=nfm&ap_tier=t50&source=listpage_search-results&order_bucket=0&topspot-algorithm=nfm-default&topspot-dealer-id=36549812'
my_url02 = 'https://www.autoscout24.de/angebote/bmw-116-i-lim-5-trg-klima-aux-start-stopp-benzin-schwarz-2d04da31-c437-4566-af7b-6ae9d4b16ab0?sort=standard&desc=0&lastSeenGuidPresent=false&cldtidx=1&position=1&search_id=rlsjwof3ir&source_otp=nfm&ap_tier=t50&source=listpage_search-results&order_bucket=0&topspot-algorithm=nfm-default&topspot-dealer-id=5401905'

page = urlopen(my_url02)
html = page.read().decode('utf-8')

In [3]:
soup = BeautifulSoup(html, 'html.parser')

In [8]:
wanted_keys = [
    'Karosserieform', 'Fahrzeugart', 'Sitzplätze', 'Türen', 'Kilometerstand', 
    'Erstzulassung', 'Baujahr', 'Scheckheftgepflegt', 'Nichtraucherfahrzeug', 'Leistung', 
    'Getriebe', 'Hubraum', 'Gänge', 'Zylinder', 'Leergewicht', 
    'Kraftstoff', 'Kraftstoffverbrauch2,8', 'CO₂-Emissionen2,8', 'Schadstoffklasse', 'Umweltplakette',
    'Komfort', 'Unterhaltung/Media', 'Sicherheit', 'Extras', 'Außenfarbe']

car = {} 
for key, value in zip(soup.find_all('dt'), soup.find_all('dd')):
    if key.text in wanted_keys:
        # join list items to a space separated string 
        list_items = value.find_all('li')
        if len(list_items) > 0:
            car_property = ' '.join([item.get_text() for item in list_items]) 
        else:
            car_property = value.text
        if key.text == 'Kraftstoffverbrauch2,8':
            category = 'Kraftstoffverbrauch'
        elif key.text == 'CO₂-Emissionen2,8':
            category = 'CO2-Emissionen'
        else:
            category = key.text
        
        # remove units
        if category == 'Kilometerstand':
            car_property = car_property[:-3].replace('.', '')
        elif category == 'Hubraum':
            car_property = car_property[:-4].replace('.','')
            
        car[category] = car_property

for k,v in car.items():
    print(f'{k}: {v}')


Karosserieform: Kombi
Fahrzeugart: Gebraucht
Sitzplätze: 5
Türen: 4
Kilometerstand: 88100
Erstzulassung: 10/2016
Scheckheftgepflegt: Ja
Nichtraucherfahrzeug: Ja
Leistung: 80 kW (109 PS)
Getriebe: Schaltgetriebe
Hubraum: 1499
Kraftstoff: Normal/Benzin 91
Kraftstoffverbrauch: 5,4 l/100 km (komb.)6,8 l/100 km (innerorts)4,6 l/100 km (außerorts)
CO2-Emissionen: 128 g/km (komb.)
Schadstoffklasse: Euro 6
Umweltplakette: 4 (Grün)
Komfort: Elektr. Fensterheber Elektrische Seitenspiegel Klimaanlage Lederlenkrad Regensensor Start/Stop-Automatik
Unterhaltung/Media: CD Radio
Sicherheit: ABS Beifahrerairbag ESP Fahrerairbag Isofix Seitenairbag Servolenkung Tagfahrlicht Traktionskontrolle Wegfahrsperre Zentralverriegelung
Extras: Alufelgen Sommerreifen
Außenfarbe: Schwarz
